In [1]:
import random
from enum import Enum
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional, Set
import tkinter as tk
from tkinter import ttk

In [2]:
class Suit(Enum):
    SPADES = "♠"
    HEARTS = "♥"
    DIAMONDS = "♦"
    CLUBS = "♣"

class Rank(Enum):
    SEVEN = "7"
    EIGHT = "8"
    NINE = "9"
    TEN = "10"
    JACK = "J"
    QUEEN = "Q"
    KING = "K"
    ACE = "A"


In [3]:
@dataclass
class Card:
    suit: Suit
    rank: Rank
    
    def __str__(self):
        return f"{self.rank.value}{self.suit.value}"

In [4]:
class Player:
    def __init__(self, position: int):
        self.position = position
        self.hand: List[Card] = []
        self.team = None  # Will be set when added to a team
        self.tricks_won = 0
        
    def receive_card(self, card: Card):
        self.hand.append(card)
        
    def play_card(self, card: Card):
        self.hand.remove(card)
        return card
    
    def get_legal_moves(self, trick) -> List[Card]:
        # Logic to determine legal moves based on the current trick
        # This is a placeholder - implement actual rules
        return self.hand
    
    def get_teammate(self):
        """Returns the teammate of this player"""
        if self.team:
            return next((p for p in self.team.players if p != self), None)
        return None
    
    def get_opponents(self):
        """Returns a list of opponent players"""
        opponents = []
        for team in self.team.game.teams:
            if team != self.team:
                opponents.extend(team.players)
        return opponents



In [5]:
class CoincheDeck:
    def __init__(self):
        self.cards = [Card(suit, rank) 
                     for suit in Suit 
                     for rank in Rank]
        
    def shuffle(self):
        random.shuffle(self.cards)
        
    def deal(self, players: List[Player], deal_pattern: Tuple[int, int, int]):
        card_index = 0
        for num_cards in deal_pattern:
            for player in players:
                for _ in range(num_cards):
                    if card_index < len(self.cards):
                        player.receive_card(self.cards[card_index])
                        card_index += 1

In [6]:
class Team:
    def __init__(self, team_id: int):
        self.team_id = team_id
        self.score = 0
        self.players = []  # Will contain references to Player objects
    
    def add_player(self, player):
        self.players.append(player)
        player.team = self
    
    def get_players(self):
        return self.players
        
    def get_score(self):
        return self.score
    
    def add_score(self, points: int):
        self.score += points



In [7]:
class CoinchePainterGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Coinche Game")
        self.canvas = tk.Canvas(root, width=800, height=600)
        self.canvas.pack()
        
    def draw_game_state(self, game_state):
        self.canvas.delete("all")
        # Draw the game board
        self.draw_board()
        
        # Draw players' hands and info
        if game_state and 'players' in game_state:
            for player in game_state['players']:
                self.draw_player(player, game_state)
        
        # Draw current trick
        if game_state and 'current_trick' in game_state and game_state['current_trick']:
            self.draw_trick(game_state['current_trick'])
        
        # Draw bidding info
        if game_state and 'current_bid' in game_state and game_state['current_bid']:
            self.draw_bid(game_state['current_bid'])
        
        # Draw team scores
        if game_state and 'teams' in game_state:
            self.draw_scores(game_state['teams'])
    
    def draw_board(self):
        # Draw the basic board layout
        self.canvas.create_rectangle(50, 50, 750, 550, outline="green", fill="darkgreen", width=2)
        
    def draw_player(self, player, game_state):
        # Draw player's hand and information based on position
        pos = player.position
        teammate = player.get_teammate()
        
        # Calculate position based on player number
        positions = [
            (400, 520),  # Bottom (player 0)
            (50, 300),   # Left (player 1)
            (400, 80),   # Top (player 2)
            (750, 300)   # Right (player 3)
        ]
        
        x, y = positions[pos]
        
        # Draw player indicator
        team_color = "blue" if player.team.team_id == 0 else "red"
        self.canvas.create_oval(x-20, y-20, x+20, y+20, fill=team_color)
        
        # Draw player's position and team info
        self.canvas.create_text(x, y-30, text=f"Player {pos} (Team {player.team.team_id})")
        if teammate:
            self.canvas.create_text(x, y-50, text=f"Teammate: Player {teammate.position}")
        
        # Draw hand for the bottom player only (to represent the viewing player)
        if pos == 0:  # Bottom player
            self.draw_hand(player.hand, x-150, y+30)
    
    def draw_hand(self, hand, x, y):
        # Draw the player's cards
        for i, card in enumerate(hand):
            color = "red" if card.suit in [Suit.HEARTS, Suit.DIAMONDS] else "black"
            self.canvas.create_rectangle(x + i*30, y, x + i*30 + 25, y + 40, fill="white")
            self.canvas.create_text(x + i*30 + 12, y + 20, text=str(card), fill=color)
    
    def draw_trick(self, trick):
        # Draw the current trick in the middle of the board
        center_x, center_y = 400, 300
        positions = [
            (center_x, center_y+60),  # Bottom
            (center_x-60, center_y),   # Left
            (center_x, center_y-60),   # Top
            (center_x+60, center_y)    # Right
        ]
        
        for i, (player, card) in enumerate(trick.cards_played):
            pos = player.position
            x, y = positions[pos]
            color = "red" if card.suit in [Suit.HEARTS, Suit.DIAMONDS] else "black"
            self.canvas.create_rectangle(x-15, y-20, x+15, y+20, fill="white")
            self.canvas.create_text(x, y, text=str(card), fill=color)
    
    def draw_bid(self, bid):
        # Draw the current bid information
        x, y = 400, 200
        text = f"Bid: {bid.points} on {bid.trump_suit.value}"
        if bid.is_coinched:
            text += " (Coinched)"
        if bid.is_surcoinched:
            text += " (Surcoinched)"
        self.canvas.create_text(x, y, text=text, fill="white")
    
    def draw_scores(self, teams):
        # Draw team scores
        self.canvas.create_text(100, 30, text=f"Team Blue: {teams[0].score}", fill="blue")
        self.canvas.create_text(700, 30, text=f"Team Red: {teams[1].score}", fill="red")


In [8]:
class CoinchePainter:
    def __init__(self):
        self.root = tk.Tk()
        self.gui = CoinchePainterGUI(self.root)
    
    def update(self, game_state):
        self.gui.draw_game_state(game_state)
        self.root.update()


In [9]:
class CoincheBid:
    def __init__(self, points: int, trump_suit: Suit, player: Player):
        self.points = points
        self.trump_suit = trump_suit
        self.player = player
        self.is_coinched = False
        self.is_surcoinched = False


In [10]:
class CoincheTrick:
    NORMAL_POINTS = {
        Rank.SEVEN: 0, Rank.EIGHT: 0, Rank.NINE: 0,
        Rank.TEN: 10, Rank.JACK: 2, Rank.QUEEN: 3,
        Rank.KING: 4, Rank.ACE: 11
    }
    
    TRUMP_POINTS = {
        Rank.SEVEN: 0, Rank.EIGHT: 0, Rank.NINE: 14,
        Rank.TEN: 10, Rank.JACK: 20, Rank.QUEEN: 3,
        Rank.KING: 4, Rank.ACE: 11
    }
    
    def __init__(self, trump_suit: Optional[Suit]):
        self.cards_played: List[Tuple[Player, Card]] = []
        self.trump_suit = trump_suit
        self.leading_suit = None
        
    def play_card(self, player: Player, card: Card) -> bool:
        if not self.leading_suit:
            self.leading_suit = card.suit
        
        # Add validation logic here for legal moves
        
        self.cards_played.append((player, card))
        return True
    
    def calculate_winner(self) -> Tuple[Player, int]:
        highest_value = -1
        winning_player = None
        total_points = 0
        
        for player, card in self.cards_played:
            points = self.TRUMP_POINTS[card.rank] if card.suit == self.trump_suit \
                    else self.NORMAL_POINTS[card.rank]
            total_points += points
            
            # Determine card strength for winning
            is_trump = card.suit == self.trump_suit
            card_strength = 100 + self.TRUMP_POINTS[card.rank] if is_trump else \
                           (50 + self.NORMAL_POINTS[card.rank] if card.suit == self.leading_suit else 0)
            
            if card_strength > highest_value:
                highest_value = card_strength
                winning_player = player
            
        return winning_player, total_points


In [11]:
class CoinceGame:
    def __init__(self):
        self.deck = CoincheDeck()
        self.players = [Player(i) for i in range(4)]
        self.teams = [Team(0), Team(1)]  # Create two teams
        self.current_trick = None
        self.tricks_played = []
        self.current_bid = None
        self.current_player = 0 # Index of current player
        self.painter = CoinchePainter()
        
        # Setup teams - players 0,2 are Team 0, players 1,3 are Team 1
        self.setup_teams()
        
        # Reference to the game in players for accessing opponents
        for player in self.players:
            player.game = self
    
    def setup_teams(self):
        # Assign players to teams (0,2 -> team 0, 1,3 -> team 1)
        self.teams[0].add_player(self.players[0])
        self.teams[0].add_player(self.players[2])
        self.teams[1].add_player(self.players[1])
        self.teams[1].add_player(self.players[3])

    def shuffle_deck(self):
        # Shuffle the deck
        self.deck.shuffle()


    def deal_card(self, player_id, deal_pattern):
        # Clear players' hands before dealing so that no player has extra cards
        for player in self.players:
            player.hand = []
    
        # No need to alter the actual self.players order for shuffling.
        # The deal_card method creates a temporary dealing order based on player_id,
        # preserving team assignments and overall player order.
        # Rearrange players so that dealing starts with the specified player_id
        players_order = self.players[player_id:] + self.players[:player_id]
        # Deal cards using the provided pattern
        self.deck.deal(players_order, deal_pattern)
        
        self.current_player += 1
        if self.current_player >= 4:
            self.current_player = 0


   

# Initialisation d'un jeu de coinche

In [12]:
game = CoinceGame()

# Exploration de tout ce qu'on peut faire

Ou sont les cartes

In [13]:
game.deck.cards[:2]

[Card(suit=<Suit.SPADES: '♠'>, rank=<Rank.SEVEN: '7'>),
 Card(suit=<Suit.SPADES: '♠'>, rank=<Rank.EIGHT: '8'>)]

In [14]:
game.current_bid, game.current_trick

(None, None)

In [15]:
game.players[0].hand

[]

# On peut mélanger le jeu de cartes

In [16]:
game.shuffle_deck()
game.deck.cards[:2]

[Card(suit=<Suit.HEARTS: '♥'>, rank=<Rank.QUEEN: 'Q'>),
 Card(suit=<Suit.CLUBS: '♣'>, rank=<Rank.SEVEN: '7'>)]

In [17]:
game.current_player

0

# On peut distribuer des cartes

In [18]:
facon_de_distribuer = [(3, 3, 2), (2, 3, 3), (3, 2, 3)]
game.deal_card(game.current_player, facon_de_distribuer[0])

In [19]:
game.players[0].hand

[Card(suit=<Suit.HEARTS: '♥'>, rank=<Rank.QUEEN: 'Q'>),
 Card(suit=<Suit.CLUBS: '♣'>, rank=<Rank.SEVEN: '7'>),
 Card(suit=<Suit.CLUBS: '♣'>, rank=<Rank.TEN: '10'>),
 Card(suit=<Suit.SPADES: '♠'>, rank=<Rank.QUEEN: 'Q'>),
 Card(suit=<Suit.HEARTS: '♥'>, rank=<Rank.JACK: 'J'>),
 Card(suit=<Suit.DIAMONDS: '♦'>, rank=<Rank.NINE: '9'>),
 Card(suit=<Suit.SPADES: '♠'>, rank=<Rank.TEN: '10'>),
 Card(suit=<Suit.HEARTS: '♥'>, rank=<Rank.EIGHT: '8'>)]

In [20]:
game.current_player

1

# On peut regarder les annonces et comment les faires